In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
from utils import *


def train(model, train_loader, test_loader, criterion, optimizer, epochs, patience=5, save_path='best_model.pth'):
    """
    Train a model with early stopping and learning rate scheduling.
    :param model: PyTorch model to train
    :param train_loader: DataLoader for training data
    :param test_loader: DataLoader for testing data
    :param criterion: Loss function
    :param optimizer: Optimizer
    :param epochs: Total number of epochs to train
    :param patience: Number of epochs to wait before early stopping (default=5)
    :param save_path: Path to save the best model's weights (default='best_model.pth')
    """
    best_loss = float('inf')  # Initialize the best loss
    patience_counter = 0  # Counter for early stopping
    best_model = None  # Variable to store the best model's weights
    
    # Set up the learning rate scheduler
    scheduler = lr_scheduler.LinearLR(
        optimizer, start_factor=1.0, end_factor=0.1, total_iters=epochs
    )

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        # Training loop
        for inputs, targets in train_loader:
            # Move inputs and targets to the appropriate device
            inputs, targets = inputs.to(device), targets.to(device)

            # Adjust input dimensions for the model
            if inputs.dim() == 2:
                inputs = inputs.unsqueeze(1)
            inputs = inputs.permute(0, 2, 1)

            optimizer.zero_grad()  # Reset gradients
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Evaluate the model on the test set
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                # Adjust input dimensions for the model
                if inputs.dim() == 2:
                    inputs = inputs.unsqueeze(1)
                inputs = inputs.permute(0, 2, 1)

                outputs = model(inputs)
                loss = criterion(outputs, targets.unsqueeze(1))
                test_loss += loss.item()

        # Calculate average losses
        avg_train_loss = train_loss / len(train_loader)
        avg_test_loss = test_loss / len(test_loader)

        # Early stopping and saving the best model
        if avg_test_loss < best_loss:
            best_loss = avg_test_loss
            patience_counter = 0
            best_model = model.state_dict()
            torch.save(best_model, save_path)
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

        # Update learning rate scheduler
        scheduler.step()

    # Restore the best model
    model.load_state_dict(torch.load(save_path))

def get_predictions(model, test_loader, device):
    """
    Generate predictions from the model on the test data.
    :param model: PyTorch model for prediction
    :param test_loader: DataLoader for test data
    :param device: Device (e.g., 'cpu' or 'cuda') to run the model
    :return: Tuple of predictions and actual values (both as NumPy arrays)
    """
    model.eval()  # Set model to evaluation mode
    predictions = []
    actuals = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            # Move inputs to the appropriate device
            inputs = inputs.to(device)

            # Adjust input dimensions for the model
            if inputs.dim() == 2:
                inputs = inputs.unsqueeze(1)
            inputs = inputs.permute(0, 2, 1)

            # Generate predictions
            outputs = model(inputs)
            predictions.append(outputs.squeeze(-1).cpu())  # Convert to CPU and remove singleton dimension
            actuals.append(targets.cpu())  # Convert actual values to CPU

    # Convert lists of tensors to NumPy arrays
    predictions = torch.cat(predictions).numpy()
    actuals = torch.cat(actuals).numpy()

    return predictions, actuals

# Set a seed for reproducibility
set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters and configuration
len_cut = 24                # We use hourly data, and our window has length of 24, that is, 1 day
kernel_size = 3             # TCN kernel size
dilations = [1, 2, 4]       # TCN dilation
num_channels = [8, 16, 32]  # Number of channels on each layer
learning_rate = 0.01        
epochs = 10
batch_size = 32 
factor = 0.1                # Hyperparameter alpha, tested with from 0.1 to 0.9
K = 2                       # SSA window length, tested with from 2 to 12 (since len_cut=24)

# Replace with actual cryptocurrency time series
original_data = np.random.randn(10000)  

# Create input sequences and labels
Input = np.array([original_data[i:i + len_cut + 1] for i in range(len(original_data) - len_cut)])
label = original_data[len_cut:]

# Normalize the input data
M = np.max(Input, axis=1)
N = np.min(Input, axis=1)
input_data = (Input - N[:, None]) / (M - N)[:, None]

# Apply KDE and filter the data
f1_results = np.array([
    kde_pdf_cdf_function(input_data[i], factor)[0](input_data[i]) 
    for i in range(input_data.shape[0])
])
filtered_x = f1_results[:, :-1]
filtered_y = f1_results[:, -1]

# Apply SSA decomposition
data = np.array([SSA_decomposition(filtered_x[i], K) for i in range(filtered_x.shape[0])])
labels = filtered_y

# Prepare the data loaders
train_loader, test_loader = prepare_data(data, labels, train_ratio=0.7, batch_size=batch_size)

# Initialize the model and move it to the device
model = TCNWithAttention(
    input_size=data.shape[1],
    num_channels=num_channels,
    kernel_size=kernel_size,
    dilations=dilations
).to(device)

# Define the loss function and optimizer
criterion = torch.nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
train(model, train_loader, test_loader, criterion, optimizer, epochs)

# Get predictions and actual labels from the test set
predictions, actuals = get_predictions(model, test_loader, device)

# Calculate the total number of labels in the test set
total_label_length = get_total_label_length(test_loader)

# Generate real predictions
pred = pred_generation(original_data, predictions, factor, total_label_length).flatten()